# detect_crb_damage.ipynb

**If a ResourceExhausteError is returned when firing up the GPU, this can usually be handled by restarting the machine.**

* 2021-05-02 First version by Aubrey Moore

This notebook uses a pair of Tensorflow object detectors to measure coconut rhinoceros beetle damage in digital images.

Example usage:

    papermill detect_crb_damage.ipynb \
     '../open-camera-test/home-uog/detect_crb_damage_output.ipynb' \
    -p IMAGE_FILE_PATH '../rawdata/*.jpg' \
    -p OUTPUT_XML_PATH '../output/detected_objects.xml'

When the above command line is executed in the directory containing **detect_crb_damage.ipynb**, 
all **jpg** image files in the **IMAGE_FILE_PATH** directory will be scanned by the
object detectors and results will be saved in **OUTPUT_XML_PATH**.

2022-07-23 Added MAX_IMAGES for testing purposes

In [1]:
import os

# Set environment variables
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# import tensorflow as tf
# uncomment following lines if you are using TF2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

/home/aubreytensor1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aubreytensor1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aubreytensor1/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aubreytensor1/.local/lib/python3.6/site-packages/tensorflow/pyth

Instructions for updating:
non-resource variables are not supported in the long term


/home/aubreytensor1/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aubreytensor1/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aubreytensor1/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aubreytensor1/.local/lib/python3.6/site-pac

In [2]:
import numpy as np
import json
import ast
import cv2
import argparse
from PIL import Image
import math

import sys
sys.path.append('Mask_RCNN')
from mrcnn.config import Config
import mrcnn.model as modellib

from xml_dumper import dump_as_cvat_annotation
import skimage.io
from collections import OrderedDict
from skimage.measure import find_contours, approximate_polygon
import glob
import logging

Using TensorFlow backend.


In [3]:
tf.__version__

'1.14.0'

In [4]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 329 µs (started: 2022-09-04 06:52:41 +10:00)


In [5]:
# parameters for papermill

IMAGE_FILE_PATH = '../rawdata/*.jpg' # Path to one or more image files. Can include wildcards. See https://pymotw.com/2/glob/ for pattern matching details.
OUTPUT_XML_PATH = '../output/detected_objects.xml' # Path to output file which will contain metadata for detected objects.
MAX_IMAGES = 1000000  # maximum number of images to be processed

TYPE = 'both' # what type of models to use [both,classes,v_shape]
#SKIP_NO = 1 # int, num of frames to skip (must be >0)
#NUM_FRAMES = None # how many frames to consider?
OD_MODEL = "object-detectors/inference_data/frozen_inference_graph_5classes.pb" # path to trained detection model
CLASSES_CVAT = "object-detectors/inference_data/5classes.csv" # classes you want to use for cvat, see readme for more details.
CLASSES_TYPE = "od" # type of classes csv file [od, maskrcnn]
MASK_MODEL =  "object-detectors/inference_data/mask_rcnn_cvat_0160.h5" # path to trained maskrcnn model
OD_THRESHOLD = 0.5 # threshold for IoU
MASK_THRESHOLD = 0.5 # threshold for maskrcnn
#SURVEY_TYPE = "v_shape" # what to write in geojson [v_shape,classes]
TASK_ID = 0 # required only if you want to use this in cvat
TASK_NAME = "demo" # required only if you want to use this in cvat
DUMP_SQL = False

time: 822 µs (started: 2022-09-04 06:52:41 +10:00)


In [6]:
class ObjectDetection:
    def __init__(self, model_path):
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(model_path , 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
                config = tf.ConfigProto()
                config.gpu_options.allow_growth=True
                self.sess = tf.Session(graph=self.detection_graph, config=config)

    def get_detections(self, image_np_expanded):
        image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')
        (boxes, scores, classes, num_detections) = self.sess.run([boxes, scores, classes, num_detections], feed_dict={image_tensor: image_np_expanded})
        return boxes, scores, classes, num_detections

    @staticmethod
    def process_boxes(boxes, scores, classes, labels_mapping, threshold, width, height):
        result = {}
        for i in range(len(classes[0])):
            if classes[0][i] in labels_mapping.keys():
                if scores[0][i] >= threshold:
                    xmin = int(boxes[0][i][1] * width)
                    ymin = int(boxes[0][i][0] * height)
                    xmax = int(boxes[0][i][3] * width)
                    ymax = int(boxes[0][i][2] * height)
                    label = labels_mapping[classes[0][i]]
                    if label not in result:
                        result[label] = []
                    result[label].append([xmin,ymin,xmax,ymax])
        return result

class Segmentation:
    def __init__(self, model_path, num_c=2):
        class InferenceConfig(Config):
            # Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
            NAME = "cvat"
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            NUM_CLASSES = num_c

        config = InferenceConfig()
        #config.display()

        # Create model object in inference mode.
        self.model = modellib.MaskRCNN(mode="inference", model_dir="./output", config=config)
        # Load weights trained on MS-COCO
        self.model.load_weights(model_path, by_name=True)
        self.labels_mapping = {0:'BG', 1:'cut'}

    def get_polygons(self, images, threshold):
        res = self.model.detect(images)
        result = {}
        for r in res:
            for index, c_id in enumerate(r['class_ids']):
                if c_id in self.labels_mapping.keys():
                    if r['scores'][index] >= threshold:
                        mask = r['masks'][:,:,index].astype(np.uint8)
                        contours = find_contours(mask, 0.5)

                        # KLUDGE
                        # Handles a rare "list index out of range error" for contours[0]
                        # If the contours array is empty, a dummy contour consisting of
                        # the top left pisxel is provided.

                        if not contours:
                            print('ERROR: contour list is empty.')
                            contour = np.array([[1.0,1.0],[1.0,0.0],[0.0,0.0],[0.0,1.0],[1.0,1.0]])
                        else:
                            contour = contours[0]
                            # print(f'contour ({type(contour)}): {contour}')

                        # end of KLUDGE

                        contour = np.flip(contour, axis=1)
                        contour = approximate_polygon(contour, tolerance=2.5)
                        segmentation = contour.ravel().tolist()
                        label = self.labels_mapping[c_id]
                        if label not in result:
                            result[label] = []
                        result[label].append(segmentation)
        return result


    @staticmethod
    def process_polygons(polygons, boxes):
        """
           Check if any point of the polygon falls into any of coconot palms except for dead/non_recoverable.
        """
        def _check_inside_boxes(polygon, boxes):
            for point in polygon:
                for label, bxes in boxes.items():
                    for box in bxes:
                        if point[0] > box[0] and point[0] < box[2] and point[1] > box[1] and point[1] < box[3] and label not in ['dead','non_recoverable']:
                            # point is inside rectangle
                            return True
            return False

        result = {}
        for label_m, polys in polygons.items():
            for polygon in polys:
                p = [polygon[i:i+2] for i in range(0, len(polygon),2)]
                if _check_inside_boxes(p, boxes):
                    if label_m not in result:
                        result[label_m] = []
                    result[label_m].append(polygon)

        return result


def load_image_into_numpy(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def draw_instances(frame, boxes, masks):
    colors = {'zero':(0,255,0), 'light':(0,0,255),'medium':(255,0,0),'high':(120,120,0),'non_recoverable':(0,120,120),'cut':(0,0,0)}
    #draw boxes
    for label, bxes in boxes.items():
        for box in bxes:
            cv2.rectangle(frame, (box[0],box[1]), (box[2],box[3]), colors[label], 5)
    #draw polygons
    for label, polygons in masks.items():
        for polygon in polygons:
            p = [polygon[i:i+2] for i in range(0, len(polygon),2)]
            pts = np.array(p, np.int32)
            pts = pts.reshape((-1,1,2))
            cv2.polylines(frame, [pts], True, (0,255,255),5)
    return frame

def get_labels(classes_csv, type="od"):
    labels = []
    with open(classes_csv, "r") as f:
        data = f.readlines()
        # slogger.glob.info("class file data {}".format(data))
        for line in data[1:]:
            if type == "maskrcnn":
                if "," not in line:
                    continue
                # slogger.glob.info("classes line {}".format(line))
                label, num = line.strip().split(',')
                labels.append(('label', [('name', line.strip())]))
            else:
                if "label" not in line:
                    labels.append(('label', [('name', line.strip())]))
    return labels

time: 8.13 ms (started: 2022-09-04 06:52:41 +10:00)


In [7]:
# NEW CODE

# Initialization
################

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(funcName)s %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S%z",
    handlers=[logging.StreamHandler()])
logging.info('Starting detect_crb_damage.ipynb')


# Get a sorted list of image files

# Intialize other variables

image_files = sorted(glob.glob(IMAGE_FILE_PATH))
num_frames = len(image_files)

labels_from_csv = get_labels(CLASSES_CVAT, CLASSES_TYPE)
final_result = {'meta':{'task': OrderedDict([('id',str(TASK_ID)),
                                             ('name',str(TASK_NAME)),
                                             ('size',str(num_frames)),
                                             ('mode','interpolation'),
                                             ('start_frame', str(0)),
                                             ('stop_frame', str(num_frames-1)),
                                             ('z_order',"False"),
                                             ('labels', labels_from_csv)])},
                'frames':[]}

if TYPE == "both":
    od_model = ObjectDetection(OD_MODEL)
    seg_model = Segmentation(MASK_MODEL)
elif TYPE == "classes":
    od_model = ObjectDetection(OD_MODEL)
elif TYPE == "v_shape":
    seg_model = Segmentation(MASK_MODEL)
    
labels_mapping_od = {1:'zero',2:'light',3:'medium',4:'high',5:'non_recoverable'}

# Get size of first image in list. It is assumed that all images are the same size.

frame = cv2.imread(image_files[0])
frame_height, frame_width, channels = frame.shape

height, width = frame_height, frame_width

# Process image files
#####################

frame_no = 0
for image_file in image_files[0:MAX_IMAGES]:
    frame_no += 1
#     print(f'Image {frame_no} of {num_frames}')
    frame = cv2.imread(image_file)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_np_expanded = np.expand_dims(img, axis=0)

    od_result = {}
    result = {}
    if TYPE == "both" or TYPE == "classes":
        # run detection
        boxes, scores, classes, num_detections = od_model.get_detections(image_np_expanded)
        #normalize bounding boxes, also apply threshold
        od_result = ObjectDetection.process_boxes(boxes, scores, classes, labels_mapping_od, OD_THRESHOLD, width, height)
        if od_result:
            #print("od", od_result)
            shapes = []
            for label, boxes in od_result.items():
                for box in boxes:
                    shapes.append({'type':'rectangle','label':label,'occluded':0,'points':box})
            final_result['frames'].append({'frame':frame_no, 'width':frame_width, 'height':frame_height, 'shapes':shapes})
    if TYPE == "both" or TYPE == "v_shape":
        # run segmentation
        result = seg_model.get_polygons([img], MASK_THRESHOLD)
        #print("Result before processing: ", result)
        if TYPE == "both" or TYPE == "classes":
            # filter out false positives if boxes are available
            result = Segmentation.process_polygons(result, od_result)
            #print("Result after processing: ", result)
        if result:
            shapes = []
            for label, polygons in result.items():
                for polygon in polygons:
                    shapes.append({'type':'polygon','label':label,'occluded':0,'points':polygon})
            frame_exists = False
            for frame_ in final_result['frames']:
                if frame_['frame'] == frame_no:
                    break
            if frame_exists:
                final_result['frames']['shapes'].extend(shapes)
            else:
                final_result['frames'].append({'frame':frame_no, 'width':frame_width, 'height':frame_height, 'shapes':shapes})
        if (frame_no % 100 == 0):
            logging.info(f'Image {frame_no} of {num_frames}')
                        
#frame = draw_instances(frame, od_result, result)
dump_as_cvat_annotation(open(OUTPUT_XML_PATH, "w"), final_result)

print('FINISHED')

2022-09-04T06:52:41+1000 [INFO] <module> Starting detect_crb_damage.ipynb


2022-09-04T06:52:43+1000 [WARNING] __getattr__ From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



2022-09-04T06:52:45+1000 [WARNING] __getattr__ From Mask_RCNN/mrcnn/model.py:341: The name tf.log is deprecated. Please use tf.math.log instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2022-09-04T06:52:45+1000 [WARNING] new_func From Mask_RCNN/mrcnn/model.py:399: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


...

2022-09-04T06:52:45+1000 [WARNING] __getattr__ From Mask_RCNN/mrcnn/model.py:722: The name tf.sparse_tensor_to_dense is deprecated. Please use tf.sparse.to_dense instead.



Instructions for updating:
Use `tf.cast` instead.


2022-09-04T06:52:46+1000 [WARNING] new_func From Mask_RCNN/mrcnn/model.py:772: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


2022-09-04T06:52:51+1000 [WARNING] __getattr__ From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

2022-09-04T06:58:24+1000 [INFO] <module> Image 100 of 23842
2022-09-04T07:04:00+1000 [INFO] <module> Image 200 of 23842
2022-09-04T07:09:37+1000 [INFO] <module> Image 300 of 23842
2022-09-04T07:15:13+1000 [INFO] <module> Image 400 of 23842
2022-09-04T07:20:49+1000 [INFO] <module> Image 500 of 23842
2022-09-04T07:26:25+1000 [INFO] <module> Image 600 of 23842
2022-09-04T07:32:02+1000 [INFO] <module> Image 700 of 23842
2022-09-04T07:37:39+1000 [INFO] <module> Image 800 of 23842
2022-09-04T07:43:15+1000 [INFO] <module> Image 900 of 23842
2022-09-04T07:48:52+1000 [INFO] <module> Image 1000 of 23842
2022-09-04T07:54:28+1000 [INFO] <module> Image 1100 of 23842
2022-09-04T08:00:05+1000 [INFO] <module> Image 1200 of 23842
2022-09-04T08:05:40+1000 [INFO] <module> Image 1300 of 238

FINISHED
time: 22h 16min 33s (started: 2022-09-04 06:52:41 +10:00)
